In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import json
import matplotlib.pyplot as plt
import os
import tqdm
import pickle
from pathlib import Path
from torch.utils.data import DataLoader
from sklearn.metrics import ndcg_score
np.random.seed(0)

In [ ]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import FreqDist
import re
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

In [ ]:
save_path='/content/drive/Shareddrives/Unlimited Drive | @LicenseMarket/Recommender/Yelp/'

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/Shareddrives/Unlimited Drive | @LicenseMarket/Recommender/Yelp/Kaggle/"

In [ ]:
%cd /content/drive/Shareddrives/Unlimited Drive | @LicenseMarket/Recommender/Yelp/

/content/drive/Shareddrives/Unlimited Drive | @LicenseMarket/Recommender/Yelp/Kaggle


In [ ]:
df_review=pd.read_csv(save_path+'yelp_reviews.csv')

In [ ]:
import json
import pandas as pd
data_file = open(save_path+"Kaggle/yelp_academic_dataset_business.json")
data = []
for line in data_file:
  data.append(json.loads(line))
business_df = pd.DataFrame(data)
data_file.close()

In [ ]:
import json
import pandas as pd
data_file = open(save_path+"Kaggle/yelp_academic_dataset_tip.json")
data = []
for line in data_file:
  data.append(json.loads(line))
tip_df = pd.DataFrame(data)
data_file.close()

In [ ]:
business_df

In [ ]:
business_df.attributes= business_df.attributes.apply(lambda y: np.nan if y==None else y)

In [ ]:
business_df.dropna(axis=0,inplace=True)

In [ ]:
business_df1=business_df[business_df['is_open']==1]

In [ ]:
business_df=business_df1[business_df1['review_count']>10]

In [ ]:
business_df

In [ ]:
tip_df.drop(tip_df.index[~tip_df['business_id'].isin(business_df['business_id'].values)], inplace=True)

In [ ]:
tip_df.reset_index(inplace=True,drop=True)

In [ ]:
tip_df

,user_id,business_id,text,date,compliment_count
0,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
1,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
2,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
3,VL12EhEdT4OWqGq0nIqkzw,xODBZmX4EmlVvbqtKN7YKg,Tacos,2012-07-27 01:48:24,0
4,OttfcRxgRrYsTg9EV5Aozg,clwjLY7PdYJpe7IP9lrqEw,Order the Tortilla Soup,2014-06-17 01:20:14,0
...,...,...,...,...,...
639223,FowxkbAixI3hlREeCgIa_Q,kfNv-JZpuN6TVNSO6hHdkw,Great experience with a phenomenal food with a...,2021-05-09 23:21:10,0
639224,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
639225,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
639226,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


In [ ]:
all_tips_ids=tip_df[['business_id','user_id']].apply(tuple, axis=1)

In [ ]:
# import glob
# exist_tips=[]
# names=os.listdir(save_path+"tips_text_bert_items/")
# # print(names)
# for name in names:
#   user_names=glob.glob((save_path+"tips_text_bert_items/{}/*.json".format(name)))
#   for user_name in user_names:
#     exist_tips.append((name,user_name.split('/')[-1][:-5]))

In [ ]:
# print(len(exist_tips))

71253


In [ ]:
# df_exist_tips = pd.DataFrame(exist_tips, columns =['business_id', 'user_id'])

In [ ]:
# df_exist_tips.to_csv(save_path+'exist_tips.csv')

In [ ]:
df_exist_tips=pd.read_csv(save_path+'exist_tips.csv')

In [ ]:
exist_tips=df_exist_tips[['business_id','user_id']].apply(tuple, axis=1).values

In [ ]:
exist_tips

array([('m_kvoXITBvYDofmcKpWEmw', '0du93EkEwKuxRG_x6hqVUg'),
       ('tejDyP6IwEvcFQ8AVxKL1Q', 'qv0Jthy-A0D95FDDIUapmg'),
       ('IZMszJcSBYaVALYDqdUN6g', 'c9ZuyshW5arT17cH1Egr7A'), ...,
       ('-Or44IdY51Ukd618kikmtA', '0oWnBfUjAv7ZmTWaW8Ufdg'),
       ('-Or44IdY51Ukd618kikmtA', '37cpUoM8hlkSQfReIEBd-Q'),
       ('-Or44IdY51Ukd618kikmtA', 'LWuQdBn6vp0BFQiR2KEvbw')], dtype=object)

In [ ]:
# new_exist_tips=[]
# names=os.listdir(save_path+"tips_text_bert_items_new/")
# # print(names)
# for name in names:
#   new_exist_tips.append(name[:-5])

In [ ]:
# print(len(new_exist_tips))

In [ ]:
# with open(save_path+'new_exist_tips_names.txt','w') as new_exist_tips_file:
#   for name in new_exist_tips:
#       new_exist_tips_file.write(name+'\n')
    
#   new_exist_tips_file.flush()

In [ ]:
all_lines=[]
with open(save_path+'new_exist_tips_names.txt','r') as new_exist_tips_file:
  all_lines=new_exist_tips_file.readlines()


In [ ]:
new_exist_tips=[]
for line in all_lines:
  new_exist_tips.append(line.replace('\n',''))


In [ ]:
len(new_exist_tips)

382549

In [ ]:
import glob
exist_items_dfs=[]
names=os.listdir(save_path+"tips_bert_items_dfs/")
# print(names)
for name in names:
  exist_items_dfs.append((name[:-5]))

In [ ]:
print(len(exist_items_dfs))

50711


In [ ]:
business_ids=list(set(tip_df['business_id'].values))

In [ ]:
i=0
for bus_id in business_ids:
  if i%10==0:
    print(i)
  i+=1
  if i<34090:
    continue
  users_ids=tip_df[tip_df['business_id']==bus_id]['user_id'].values
  if bus_id in exist_items_dfs:
    df1=pd.read_json(save_path+"tips_bert_items_dfs/{}.json".format(bus_id))
  else:
    df1=pd.DataFrame()
  for user_id in users_ids:
    if bus_id+'_'+user_id in new_exist_tips:
      # print(bus_id+'_'+user_id)
      df2=pd.read_json(save_path+"tips_text_bert_items_new/{}.json".format(bus_id+'_'+user_id))
      df1=pd.concat([df1,df2],ignore_index=True)
  df1.to_json(save_path+"tips_bert_items_dfs/{}.json".format(bus_id))

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 25.9 MB/s 
     |████████████████████████████████| 163 kB 70.9 MB/s 
     |████████████████████████████████| 7.6 MB 65.5 MB/s 


In [ ]:
gpu=True
cuda='0'
if gpu:
  device = torch.device('cuda:%s' % cuda)
else:
  device = 'cpu'
print(device)

cuda:0


In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel
import torch
config = AutoConfig.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased").to(device)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
tokenizer_LM = BertTokenizer.from_pretrained("bert-base-uncased")
model_LM = BertForMaskedLM.from_pretrained("bert-base-uncased", output_hidden_states=True).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def lemmatization(text):
    result=''
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
        # if pos in ['n','a']:   
        result+=wordnet.lemmatize(token,pos)+' '
    return result
def remove_stopwords(text):
    en_stopwords = stopwords.words('english')
    en_stopwords+=['this','made','that','the','those','these','may','could','that','without','iii','with','and','This','That','Those','These','the','The','brbr','of','you']
    result = []
    for token in text:
        if token.lower() not in en_stopwords and len(token)>2:
            result.append(token)
            
    return result
def remove_punct(text):
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

def remove_tag(text):
    text=' '.join(text)
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def preprocess(text):
  chars=['&','%','#','@','^','>','<','\n','\\','\t',';','"','/']
  stwords=stopwords.words('english')
  for ch in chars:
    text=text.replace(ch,' ')
  text=" ".join(text.split())
  # text=text.lower()
  text_tokenized=word_tokenize(text)
  cleaned_text= remove_stopwords(text_tokenized)
  cleaned_text= remove_punct(cleaned_text)
  # cleaned_text=lemmatization(cleaned_text)
  cleaned_text=remove_tag(cleaned_text)
  cleaned_text=remove_urls(cleaned_text)
  cleaned_text=''.join([i for i in cleaned_text ])
  # splited=cleaned_text.split(' ')
  cleaned_text=cleaned_text.split(' ')
  new_text=''
  last_word=None
  for i in range(0,len(cleaned_text)):
    word=cleaned_text[i]
    if last_word == word:
      continue
    last_word=word
    if len(word)>1:
      new_text+=word+' '
  # print(cleaned_text)
  return new_text[:-1]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
import re
haRegex = re.compile(r'[a-z]*[A-Z]*\d+[a-z]*[A-Z]*')
def preprocess_text_first(text):
  text=text.replace('</b>','.').replace('<br />','.').replace('<b>','.')
  text=text.replace('..','.').replace('..','.').replace('..','.')
  while '<' in text and '>' in text and text.index('<')<text.index('>'):
    toRemove=text[int(text.index('<')):int(text.index('>'))]+'>'
    text=text.replace(toRemove,' ')
  list_to_replace=['nbsp','\n','mso','gte','xml','false','#',',','!','-','\'','\"','[',']','/','\\n','\\','span','a-size-base','a-color-secondary','input type','header name','value','=','<a href= javascript:void(0) class= ','{','}','class=','header','<a href= javascript:void(0)','<','>','href',')','(',';','quot','&',':','javascript']
  for char in list_to_replace:
    text=text.replace(char,' ')
  for i in range(15):
    text=text.replace('  ',' ')
  # while 'if' in text and 'endif' in text and text.index('if')<text.index('endif'):
  #   # print(int(text.index('if')),int(text.index('endif')))
  #   toRemove=text[int(text.index('if')):int(text.index('endif'))]+'endif'
  #   text=text.replace(toRemove,' ')
  new_text=''
  splited=text.split(' ')
  for i in range(0,len(splited)):
    word=splited[i]
    regFound=haRegex.search(word) 
    if len(word)>1 and len(word)<35 and regFound == None:
    # if len(word)>1 and len(word)<35:
      new_text+=word+' '
  # new_text=lemmatization(new_text)
  # print(new_text)
  return new_text[:-1]

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model=model.to(device)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_LM=model_LM.to(device)

In [ ]:
def get_replace_words(sent,word):
  words=[]
  sent=sent.lower()
  new_sent=sent.replace(word,'[MASK]')
  inputs = tokenizer_LM(new_sent, return_tensors="pt").to(device)
  mask_token_index = (inputs.input_ids == tokenizer_LM.mask_token_id)[0].nonzero(as_tuple=True)[0]
  with torch.no_grad():
      outputs = model_LM(**inputs)
  # predicted_token_id = outputs.logits[0, mask_token_index].argmax(axis=-1)
  sorted=outputs.logits[0,mask_token_index].sort(axis=-1)
  ids=sorted.indices[0][-5:]
  logits=sorted.values[0][-5:]
  all_logits=[]
  i=0
  # print(ids)
  # print(tokenizer.decode(ids[0]))
  for id in ids:
    new_word=tokenizer.decode(id).replace(' ','')
    words.append(new_word)
    all_logits.append(logits[i].tolist())
    i+=1
  if word not in words:
    ids=sorted.indices[0][-10:]
    for id in ids:
      new_word=tokenizer.decode(id).replace(' ','')
      if word == new_word:
        ind=sorted.indices[0].tolist().index(id)
        all_logits.append(sorted.values[0][ind].tolist())
        words.append(word)
        break
  if word not in words:
    all_logits.append(5.0)
    words.append(word)
  return words,all_logits

In [ ]:
tip_df.drop(['date','compliment_count'],axis=1,inplace=True)

In [ ]:
tip_df['user_bus_id']=tip_df[['business_id','user_id']].apply(tuple, axis=1)

In [ ]:
tip_df.drop(tip_df.index[tip_df['user_bus_id'].isin(exist_tips)], inplace=True)

In [ ]:
tip_df

,user_id,business_id,text,user_bus_id
709,QVoy32vmYsjcusOu5ITpDw,j-qtdD55OLfSqfsWuQTDJg,This is where I am...,"(j-qtdD55OLfSqfsWuQTDJg, QVoy32vmYsjcusOu5ITpDw)"
1045,7pG99hFW0eRH_XBvvd7xIA,bbGwoO9kvR3GqZ2Q2G8rew,Don't go,"(bbGwoO9kvR3GqZ2Q2G8rew, 7pG99hFW0eRH_XBvvd7xIA)"
1091,_ubawFAszv3RFvFRUOGIyw,9I15RdM5Dvgx-fSgC5LW8A,#2,"(9I15RdM5Dvgx-fSgC5LW8A, _ubawFAszv3RFvFRUOGIyw)"
1501,nlsxDX6nKcVDGWzVxdsWiQ,cN8F-BmoHoS9eCqIHMtlHA,Fa La La!,"(cN8F-BmoHoS9eCqIHMtlHA, nlsxDX6nKcVDGWzVxdsWiQ)"
1885,MUBoVv-Z50T4FwhZKc-z8w,6rRGA1V-mLl7f2BqqZ_AmA,.................,"(6rRGA1V-mLl7f2BqqZ_AmA, MUBoVv-Z50T4FwhZKc-z8w)"
...,...,...,...,...
639223,FowxkbAixI3hlREeCgIa_Q,kfNv-JZpuN6TVNSO6hHdkw,Great experience with a phenomenal food with a...,"(kfNv-JZpuN6TVNSO6hHdkw, FowxkbAixI3hlREeCgIa_Q)"
639224,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,"(OaGf0Dp56ARhQwIDT90w_g, 1uxtQAuJ2T5Xwa_wp7kUnA)"
639225,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,"(hYnMeAO77RGyTtIzUSKYzQ, v48Spe6WEpqehsF2xQADpg)"
639226,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,"(s2eyoTuJrcP7I_XyjdhUHQ, ckqKGM2hl7I9Chp5IpAhkw)"


In [ ]:
def get_replace_vects(sent,word,replaced_words):
  sent=sent.lower()
  vects=[]
  for new_word in replaced_words:
    new_sent=sent.replace(word,new_word)
    inputs = tokenizer(new_sent, return_tensors="pt").to(device)
    outputs = model(**inputs)
    hiddens=outputs.last_hidden_state
    tokenized=tokenizer.tokenize(new_sent)
    if new_word in tokenized:
      ind=tokenized.index(new_word)
      vects.append(list(hiddens[0][ind+1].detach().cpu().numpy()))
    else:
      vects.append(None)
  return vects

In [ ]:
k=0 
with open(save_path+'new_exist_tips_names.txt','a') as new_exist_tips_file:
  for i, row in tip_df.iterrows():
    dict_bert1={'logit':[],'replaced_word':[],'word':[],'vect':[]}
    if k%200==0:
      print(k)
    k+=1
    if k<400000:
      continue
    allOk=True
    business_id=row['business_id']
    user_id=row['user_id']
    if business_id+'_'+user_id in new_exist_tips:
      continue
    text=row['text']
    text1=preprocess_text_first(str(text))
    sents_text= text1.split('.')
    for sent in sents_text:
        if len(sent.split(' '))>512  or len(sent)<3:
          continue
        if len(sent.split(' '))>400:
          sent=' '.join(list(set(sent.split(' '))))
        new_sent=preprocess(sent).lower()
        # print(sent)
        # print(new_sent)
        tokenized_old=tokenizer.tokenize(sent)
        tokenized_new=tokenizer.tokenize(new_sent)
        if len(tokenized_old)>=512:
          continue
        # print(tokenized_old)
        # print(tokenized_new)
        # print(len(tokenized_old))
        inputs1 = tokenizer(sent, return_tensors="pt").to(device)
        outputs1 = model(**inputs1)
        hiddens1=outputs1.last_hidden_state
        not_in_vocab1=[]
        for word in new_sent.split(' '):
          if word not in tokenizer.vocab.keys():
            not_in_vocab1.append(word)

        for i in range(len(tokenized_old)):
          if tokenized_old[i] != '[UNK]' and tokenized_old[i] in tokenized_new  and tokenized_old[i] in new_sent.split(' '):
            # dict_bert1['description_words'].append(tokenized_old[i])
            # dict_bert1['description_vects'].append(list(hiddens1[0][i+1].detach().cpu().numpy()))
            # dict_bert1['item_id'].append(item_id)
            replaced_words,logits=get_replace_words(sent,tokenized_old[i])
            # print(tokenized_old[i],replaced_words,logits)
            vects_replaced_words=get_replace_vects(sent,tokenized_old[i],replaced_words)
            for j in range(len(replaced_words)):
              if vects_replaced_words[j]!=None:
                dict_bert1['word'].append(tokenized_old[i])
                dict_bert1['replaced_word'].append(replaced_words[j])
                dict_bert1['logit'].append(logits[j])
                dict_bert1['vect'].append(vects_replaced_words[j])
              
        for word in not_in_vocab1:
          tokenized_word=tokenizer.tokenize(word)
          vect1=[]
          #  print(word)
          #  print(len(word))
          for i in range(len(tokenized_old)):
              if tokenized_old[i] in tokenized_word:
                vect1.append(list(hiddens1[0][i+1].detach().cpu().numpy()))
          if len(vect1)>0:
            dict_bert1['word'].append(word)
            dict_bert1['replaced_word'].append(word)
            dict_bert1['logit'].append(12.0)
            dict_bert1['vect'].append(list(np.average(vect1,axis=0))) 

    if len(dict_bert1['vect'])==0:
      # print(user_id,business_id)
      allOk=False
      continue
    
    dframe_bert1=pd.DataFrame.from_dict(dict_bert1)
    if business_id in exist_items_dfs:
      # print(business_id)
      try:
        df1=pd.read_json(save_path+"tips_bert_items_dfs/{}.json".format(business_id))
      except ValueError:
        print(business_id)
        df1=pd.DataFrame()
    else:
      df1=pd.DataFrame()
    df1=pd.concat([df1,dframe_bert1],ignore_index=True)
    df1.to_json(save_path+"tips_bert_items_dfs/{}.json".format(business_id))
    # dframe_bert1.to_json(save_path+'tips_text_bert_items_new/'+'{}_{}.json'.format(business_id,user_id))
    new_exist_tips_file.write(business_id+'_'+user_id+'\n')
    new_exist_tips_file.flush()
    # dframe_bert1.to_json(save_path+'tips_text_bert_users_new/'+'{}_{}.json'.format(user_id,business_id))

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600
12800
13000
13200
13400
13600
13800
14000
14200
14400
14600
14800
15000
15200
15400
15600
15800
16000
16200
16400
16600
16800
17000
17200
17400
17600
17800
18000
18200
18400
18600
18800
19000
19200
19400
19600
19800
20000
20200
20400
20600
20800
21000
21200
21400
21600
21800
22000
22200
22400
22600
22800
23000
23200
23400
23600
23800
24000
24200
24400
24600
24800
25000
25200
25400
25600
25800
26000
26200
26400
26600
26800
27000
27200
27400
27600
27800
28000
28200
28400
28600
28800
29000
29200
29400
29600
29800
30000
30200
30400
30600
30800
31000
31200
31400
31600
31800
32000
32200
32400
32600
32800
33000
33200
33400
33600
33800
34000
34200
34400
34600
34800
35000
3